In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [5]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [6]:
sen_df = spark.createDataFrame([
   (0, "Hi I heard about Spark"),
   (1, "I wish Java could use case classes"),
   (2, "Logistic,regression,models,are,neat") 
], ['id','sentence'])

In [7]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [8]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [9]:
regex_tokenizer = RegexTokenizer(inputCol = 'sentence', outputCol='words', pattern='\\W') 

In [10]:
count_token = udf(lambda words: len(words), IntegerType())

In [11]:
tokenized = tokenizer.transform(sen_df)

In [12]:
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish Java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [14]:
tokenized.withColumn('token', count_token(col('words'))).show()

+---+--------------------+--------------------+-----+
| id|            sentence|               words|token|
+---+--------------------+--------------------+-----+
|  0|Hi I heard about ...|[hi, i, heard, ab...|    5|
|  1|I wish Java could...|[i, wish, java, c...|    7|
|  2|Logistic,regressi...|[logistic,regress...|    1|
+---+--------------------+--------------------+-----+



In [15]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [16]:
rg_tokenized.withColumn('token', count_token(col('words'))).show()

+---+--------------------+--------------------+-----+
| id|            sentence|               words|token|
+---+--------------------+--------------------+-----+
|  0|Hi I heard about ...|[hi, i, heard, ab...|    5|
|  1|I wish Java could...|[i, wish, java, c...|    7|
|  2|Logistic,regressi...|[logistic,regress...|    1|
+---+--------------------+--------------------+-----+



In [17]:
from pyspark.ml.feature import StopWordsRemover

In [21]:
sentenceData = spark.createDataFrame([
    (0, ["I", "saw", "the", "red", "balloon"]),
    (1, ["Mary", "had", "a", "little", "lamb"])
], ["id", "tokens"])

In [22]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [23]:
remover.transform(sentenceData).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, red...| [saw, red, balloon]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [24]:
# n-gram

In [25]:
from pyspark.ml.feature import NGram

In [26]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])


In [27]:
ngram = NGram(n=2, inputCol='words', outputCol = 'grams')

In [28]:
ngram.transform(wordDataFrame).show()

+---+--------------------+--------------------+
| id|               words|               grams|
+---+--------------------+--------------------+
|  0|[Hi, I, heard, ab...|[Hi I, I heard, h...|
|  1|[I, wish, Java, c...|[I wish, wish Jav...|
|  2|[Logistic, regres...|[Logistic regress...|
+---+--------------------+--------------------+



In [29]:
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+

